In [7]:
class HTML:
    def __init__(self, output):
        self.output = output
        self.items = []
            
    def __iadd__(self, other):
        self.items.append(other)
        return self        

    def __str__(self):
        child = ""
        for item in self.items:
            child += str(item)
        html = f"<html>\n{child}</html>"
        return html
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        if self.output!=None:
            with open(self.output, "w") as file:
                file.write(str(self))
        else:
            print(str(self))

            
class Tag:
    def __init__(self, tag, is_single=False, klass = None, **attribs):
        self.tag = tag
        self.text = ""
        self.klass = ""
        self.otherattrs = ""
        self.items = []
        self.attrib = {}
        self.is_single = is_single
        
        if klass is not None:
            self.klass = f' class = "{" ".join(list(klass))}"'
        
        for key, value in attribs.items():
            self.otherattrs = f' {key} = "{value}"'
            
    def __iadd__(self, other):
        self.items.append(other)
        return self            
            
    def __enter__(self):
        return self

    def __exit__(self, *args, **kwargs):
        pass

    def __str__(self):
        if self.is_single:
            return f'<{self.tag}{self.klass}{self.otherattrs}/>\n'
        else:
            if self.text:
                return f'<{self.tag}{self.klass}{self.otherattrs}>{self.text}<{self.tag}/>\n'
            elif self.text=="" and self.items==[]:
                return f'<{self.tag}{self.klass}{self.otherattrs}><{self.tag}/>\n'
            else:
                child = ""
                for item in self.items:
                    child += f"{str(item)}"
                tabbed_child =""
                for line in child.split("\n")[0:-1]:
                    tabbed_child += f"    {line}\n"
                child = tabbed_child
                child = "    "+"\n    ".join(child.split("\n")[0:-1])+ "\n"
                tagtext = f"<{self.tag}>\n{child}</{self.tag}>\n"
                return tagtext            
            
class TopLevelTag:
    def __init__(self, tag):
        self.tag = tag
        self.items = []
        
    def __iadd__(self, other):
        self.items.append(other)
        return self
    
    def __str__(self):
        child = ""
        for item in self.items:
            child += f"{str(item)}"
        tabbed_child =""
        for line in child.split("\n")[0:-1]:
            tabbed_child += f"    {line}\n"
        child = tabbed_child    
        tagtext = f"<{self.tag}>\n{child}</{self.tag}>\n"
        return tagtext
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args, **kwargs):
        pass           

In [8]:
if __name__ == "__main__":
    with HTML(output="test1.html") as doc:
        with TopLevelTag("head") as head:
            with Tag("title") as title:
                title.text = "hello"
                head += title
            doc += head

        with TopLevelTag("body") as body:
            with Tag("h1", klass=("main-text",)) as h1:
                h1.text = "Test"
                body += h1

            with Tag("div", klass=("container", "container-fluid"), id="lead") as div:
                with Tag("p") as paragraph:
                    paragraph.text = "another test"
                    div += paragraph

                with Tag("img", is_single=True, src="/icon.png") as img:
                    div += img

                body += div

            doc += body